- 데이터 전처리, 종결어미 전환을 거친 데이터 aihub_convert의 형태소 분석을 진행합니다. 
- 원문-생성요약문(aihub_ori_abs.csv), 원문-추출요약문(aihub_ori_ext.csv), 추출요약문-생성요약문(aihub_ext_abs.csv) 세 가지 데이터 쌍을 생성합니다. 
- 각 데이터쌍은 형태소분석 및 품사태깅 정보를 담은 컬럼을 포함합니다. 

In [1]:
# #- Install dependencies
# ! Install Java 1.8 or up
# ! yes | sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl

# #- Install KoNLPy
# ! python3 -m pip install --upgrade pip
# ! python3 -m pip install konlpy       # Python 3.x

# #- mecab 설치
# ! yes | sudo apt-get install curl git
# ! yes | bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [2]:
%%time

import os
import csv
import pandas as pd

file_name = os.getenv('HOME')+'/aiffel/aeumgil/AIhub/data/aihub_convert.csv'
df = pd.read_csv(file_name)

CPU times: user 12.7 s, sys: 1.84 s, total: 14.6 s
Wall time: 29.1 s


In [3]:
df.head()

,title,original_text,converted_ext,converted_abs,size,ori_char_cnt,abs_char_cnt,ext_char_cnt,readable,accurate,informative,trustworthy,media_sub_type
0,"논 타작물 재배, 2월 말까지 신청하세요",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,small,927,155,190,4,3,3,3,지역지
1,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성",여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,여수시는 컬러빌리지 사업에 8억원을 투입하여 색채와 빛 도시를 완성하여 고소천사벽화...,small,764,118,182,4,4,5,4,지역지
2,“새해 정기 받고 올해는 반드시 일내자!”,전남드래곤즈(사장 신승재)는 지난 4일 구봉산 해맞이 행사를 통해 새해 각오를 다졌...,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,medium,1066,68,216,4,4,4,4,지역지
3,"농업인 역량 강화, 새해 실용교육 실시",광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,"광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...",small,746,68,211,4,4,5,4,지역지
4,타이완 크루즈관광객 4천여명‘전남’온다,타이완의 크루즈관광객 4000여명이 올해 두 차례에 걸쳐 여수에 입항한다. 전라남도...,전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급 아쿠아리우스 호가...,올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,small,978,66,257,4,4,4,4,지역지


In [4]:
from konlpy.tag import Mecab

mecab = Mecab()

In [5]:
pos_dict = {'EC':'연결어미','EF':'종결어미','SF':'마침표','NNG':'일반명사','NNP':'고유명사','SN':'숫자','MAG':'일반부사','MAJ':'접속부사'}

def morph(x): return mecab.morphs(x)
def pos(x): return mecab.pos(x)
def only_pos(x): return [i[1] for i in x]
def num(x): return len(x)

def EC(x): return [i[0] for i in x if 'EC' in i[1]] #- 연결어미
def EF(x): return [i[0] for i in x if 'EF' in i[1]] #- 종결어미
def SF(x): return [i[0] for i in x if 'SF' in i[1]] #- 마침표
def NNG(x): return [i[0] for i in x if 'NNG' in i[1]] #- 일반명사
def NNP(x): return [i[0] for i in x if 'NNP' in i[1]] #- 고유명사
def SN(x): return [i[0] for i in x if 'SN' in i[1]] #- 숫자
def MAG(x): return [i[0] for i in x if 'MAG' in i[1]] #- 일반부사
def MAJ(x): return [i[0] for i in x if 'MAJ' in i[1]] #- 접속부사

In [6]:
# %%time 

#- 원문 형태소분석/ 개수
# df['ori_morph'] = df['original_text'].map(lambda x: morph(x) )
# df['ori_morph_cnt'] = df['ori_morph'].map(lambda x: num(x) )

# #- 추출요약문 형태소분석/ 개수
# df['ext_morph'] = df['converted_ext'].map(lambda x: morph(x) )
# df['ext_morph_cnt'] = df['ext_morph'].map(lambda x: num(x) )

# #- 생성요약문 형태소분석/ 개수
# df['sum_morph'] = df['converted_abs'].map(lambda x: morph(x) )
# df['sum_morph_cnt'] = df['sum_morph'].map(lambda x: num(x) )

In [7]:
#-  원문의 품사분석/ 품사만 추출
df['ori_pos'] = df['original_text'].map(lambda x: pos(x) )
df['only_ori_pos'] = df['ori_pos'].map(lambda x: only_pos(x))
df['ori_morph_cnt'] = df['only_ori_pos'].map(lambda x: num(x) )

In [8]:
%%time

#-  추출요약문의 품사분석/ 품사만 추출
df['ext_pos'] = df['converted_ext'].map(lambda x: pos(x) )
df['only_ext_pos'] = df['ext_pos'].map(lambda x: only_pos(x))
df['ext_morph_cnt'] = df['only_ext_pos'].map(lambda x: num(x) )


#-  추출요약문의 품사분석/ 품사만 추출
df['abs_pos'] = df['converted_abs'].map(lambda x: pos(x) )
df['only_abs_pos'] = df['abs_pos'].map(lambda x: only_pos(x))
df['abs_morph_cnt'] = df['only_abs_pos'].map(lambda x: num(x) )


CPU times: user 4min 36s, sys: 6.7 s, total: 4min 43s
Wall time: 4min 43s


In [9]:
df.head()

,title,original_text,converted_ext,converted_abs,size,ori_char_cnt,abs_char_cnt,ext_char_cnt,readable,accurate,...,media_sub_type,ori_pos,only_ori_pos,ori_morph_cnt,ext_pos,only_ext_pos,ext_morph_cnt,abs_pos,only_abs_pos,abs_morph_cnt
0,"논 타작물 재배, 2월 말까지 신청하세요",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,small,927,155,190,4,3,...,지역지,"[(전라남도, NNP), (가, JKS), (쌀, NNG), (과잉, NNG), (...","[NNP, JKS, NNG, NNG, NNG, JKO, NNG, XSN, JKB, ...",419,"[(전라남도, NNP), (가, JKS), (쌀, NNG), (과잉, NNG), (...","[NNP, JKS, NNG, NNG, NNG, JKO, NNG, XSN, JKB, ...",97,"[(전라남도, NNP), (가, JKS), (쌀, NNG), (과잉, NNG), (...","[NNP, JKS, NNG, NNG, NNG, JKO, NNG, XSN, JKB, ...",81
1,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성",여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,여수시는 컬러빌리지 사업에 8억원을 투입하여 색채와 빛 도시를 완성하여 고소천사벽화...,small,764,118,182,4,4,...,지역지,"[(여수시, NNP), (는, JX), (원, NNG), (도심, NNG), (일대...","[NNP, JX, NNG, NNG, NNG, JKB, NNG, XSV+ETM, NN...",339,"[(여수시, NNP), (는, JX), (원, NNG), (도심, NNG), (일대...","[NNP, JX, NNG, NNG, NNG, JKB, NNG, XSV+ETM, NN...",90,"[(여수시, NNP), (는, JX), (컬러, NNG), (빌리지, NNG), (...","[NNP, JX, NNG, NNG, NNG, JKB, SN, NR, NNBC, JK...",61
2,“새해 정기 받고 올해는 반드시 일내자!”,전남드래곤즈(사장 신승재)는 지난 4일 구봉산 해맞이 행사를 통해 새해 각오를 다졌...,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,medium,1066,68,216,4,4,...,지역지,"[(전남, NNP), (드래곤즈, NNP), ((, SSO), (사장, NNG), ...","[NNP, NNP, SSO, NNG, NNP, SSC, JX, VV+ETM, SN,...",454,"[(임직원, NNG), (과, JC), (선수단, NNG), (모두, NNG), (...","[NNG, JC, NNG, NNG, JX, NNG, NNP, NNG, JKB, VV...",99,"[(전남, NNP), (드래곤즈, NNP), (임직원, NNG), (과, JC), ...","[NNP, NNP, NNG, JC, NNG, JKS, SN, NNBC, NNP, N...",29
3,"농업인 역량 강화, 새해 실용교육 실시",광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,"광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...",small,746,68,211,4,4,...,지역지,"[(광양시, NNP), (는, JX), (오, VV), (는, ETM), (11, ...","[NNP, JX, VV, ETM, SN, NNBC, JX, SN, NNBC, JX,...",325,"[(광양시, NNP), (는, JX), (오, VV), (는, ETM), (11, ...","[NNP, JX, VV, ETM, SN, NNBC, JX, SN, NNBC, JX,...",100,"[(광양시, NNP), (는, JX), (농업, NNG), (인, XSN), (들,...","[NNP, JX, NNG, XSN, XSN, JKG, NNG, JKO, VV, EC...",35
4,타이완 크루즈관광객 4천여명‘전남’온다,타이완의 크루즈관광객 4000여명이 올해 두 차례에 걸쳐 여수에 입항한다. 전라남도...,전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급 아쿠아리우스 호가...,올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,small,978,66,257,4,4,...,지역지,"[(타이완, NNP), (의, JKG), (크루즈, NNP), (관광객, NNG),...","[NNP, JKG, NNP, NNG, SN, XSN, NNBC, JKS, NNG, ...",402,"[(전라남도, NNP), (는, JX), (올해, NNG), (4, SN), (월,...","[NNP, JX, NNG, SN, NNBC, JC, SN, NNBC, NNP, NN...",113,"[(올해, NNG), (4, SN), (월, NNBC), (과, JC), (6, S...","[NNG, SN, NNBC, JC, SN, NNBC, MM, NNG, JKB, VV...",32


In [10]:
# df.to_csv('aihub_pos', index=False)

In [11]:
%%time
#- 원문의 품사 및 문장길이(형태소) 컬럼 생성

for i in pos_dict.keys():
    df['ori_'+i] = df['ori_pos'].map(lambda x: eval(i)(x)) 
    df['ori_'+i+'_cnt'] = df['ori_'+i].map(lambda x: num(x))

CPU times: user 2min 16s, sys: 1.11 s, total: 2min 18s
Wall time: 2min 17s


In [12]:
%%time
#- 추출요약문의 품사 및 문장길이(형태소) 컬럼 생성

for i in pos_dict.keys():
    df['ext_'+i] = df['ext_pos'].map(lambda x: eval(i)(x)) 
    df['ext_'+i+'_cnt'] = df['ext_'+i].map(lambda x: num(x))

CPU times: user 58.7 s, sys: 704 ms, total: 59.4 s
Wall time: 59.4 s


In [13]:
%%time
#- 생성요약문의 품사 및 문장길이(형태소) 컬럼 생성

for i in pos_dict.keys():
    df['abs_'+i] = df['abs_pos'].map(lambda x: eval(i)(x)) 
    df['abs_'+i+'_cnt'] = df['abs_'+i].map(lambda x: num(x))

CPU times: user 36.4 s, sys: 636 ms, total: 37.1 s
Wall time: 37 s


In [22]:
%%time

#- 원문과 생성요약문의 길이(형태소 개수) 차이
df['de_ori2abs_morph'] = df['ori_morph_cnt'] - df['abs_morph_cnt']

#- 원문과 추출요약문의 길이(형태소 개수) 차이
df['de_ori2ext_morph'] = df['ori_morph_cnt'] - df['ext_morph_cnt']

#- 추출요약문과 생성요약문의 길이(형태소 개수) 차이
df['de_ext2abs_morph'] = df['ext_morph_cnt'] - df['abs_morph_cnt']

CPU times: user 5.5 ms, sys: 34 µs, total: 5.54 ms
Wall time: 4.77 ms


In [23]:
df.columns

Index(['title', 'original_text', 'converted_ext', 'converted_abs', 'size',
       'ori_char_cnt', 'abs_char_cnt', 'ext_char_cnt', 'readable', 'accurate',
       'informative', 'trustworthy', 'media_sub_type', 'ori_pos',
       'only_ori_pos', 'ori_morph_cnt', 'ext_pos', 'only_ext_pos',
       'ext_morph_cnt', 'abs_pos', 'only_abs_pos', 'abs_morph_cnt', 'ori_EC',
       'ori_EC_cnt', 'ori_EF', 'ori_EF_cnt', 'ori_SF', 'ori_SF_cnt', 'ori_NNG',
       'ori_NNG_cnt', 'ori_NNP', 'ori_NNP_cnt', 'ori_SN', 'ori_SN_cnt',
       'ori_MAG', 'ori_MAG_cnt', 'ori_MAJ', 'ori_MAJ_cnt', 'ext_EC',
       'ext_EC_cnt', 'ext_EF', 'ext_EF_cnt', 'ext_SF', 'ext_SF_cnt', 'ext_NNG',
       'ext_NNG_cnt', 'ext_NNP', 'ext_NNP_cnt', 'ext_SN', 'ext_SN_cnt',
       'ext_MAG', 'ext_MAG_cnt', 'ext_MAJ', 'ext_MAJ_cnt', 'abs_EC',
       'abs_EC_cnt', 'abs_EF', 'abs_EF_cnt', 'abs_SF', 'abs_SF_cnt', 'abs_NNG',
       'abs_NNG_cnt', 'abs_NNP', 'abs_NNP_cnt', 'abs_SN', 'abs_SN_cnt',
       'abs_MAG', 'abs_MAG_cnt', 'abs_MAJ

In [24]:
tmp = {'original_text':'ori','converted_ext':'ext','converted_abs':'abs'}
original = []; extractive = []; abstractive = []

for i,j in tmp.items():
    globals()[f'{i}'] = [i, j+'_char_cnt', j+'_pos', 'only_'+j+'_pos', j+'_morph_cnt'] + [j+'_'+p for p in pos_dict.keys()]
    
score = ['readable','accurate','informative','trustworthy']
extra = ['title','media_sub_type','size']

In [25]:
ori_abs = original_text + converted_abs + score + extra + ['de_ori2abs_morph']
ori_ext = original_text + converted_ext + score + extra + ['de_ori2ext_morph'] 
ext_abs = converted_ext + converted_abs + score + extra + ['de_ext2abs_morph']

In [26]:
#- 원문-생성요약문 데이터
ori_abs_df = df[ori_abs]
ori_abs_df.head()

,original_text,ori_char_cnt,ori_pos,only_ori_pos,ori_morph_cnt,ori_EC,ori_EF,ori_SF,ori_NNG,ori_NNP,...,abs_MAG,abs_MAJ,readable,accurate,informative,trustworthy,title,media_sub_type,size,de_ori2abs_morph
0,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,927,"[(전라남도, NNP), (가, JKS), (쌀, NNG), (과잉, NNG), (...","[NNP, JKS, NNG, NNG, NNG, JKO, NNG, XSN, JKB, ...",419,"[위해, 벼, 으면, 해, 나, 해, 해야, 도와, 으나, 다, 며, 도록, 해, ...","[다, 다, 다, 다, 된다, 다, 다, 된다, 한다, 다, 다]","[., ., ., ., ., ., ., ., ., ., .]","[쌀, 과잉, 문제, 근본, 해결, 올해, 시행, 쌀, 생산, 조정, 적극, 추진,...","[전라남도, 전남, 전남, 최향, 전라남도]",...,[],[],4,3,3,3,"논 타작물 재배, 2월 말까지 신청하세요",지역지,small,338
1,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,764,"[(여수시, NNP), (는, JX), (원, NNG), (도심, NNG), (일대...","[NNP, JX, NNG, NNG, NNG, JKB, NNG, XSV+ETM, NN...",339,"[며, 면, 따라, 면, 어, 통해, 고, 해, 고, 돼, 고, 고, 해, 어, 어...","[다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다]","[., ., ., ., ., ., ., ., ., ., .]","[원, 도심, 일대, 추진, 컬러, 빌리지, 사업, 지난해, 말, 마무리, 색채, ...",[여수시],...,[알록달록],[],4,4,5,4,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성",지역지,small,278
2,전남드래곤즈(사장 신승재)는 지난 4일 구봉산 해맞이 행사를 통해 새해 각오를 다졌...,1066,"[(전남, NNP), (드래곤즈, NNP), ((, SSO), (사장, NNG), ...","[NNP, NNP, SSO, NNG, NNP, SSC, JX, VV+ETM, SN,...",454,"[통해, 올라, 며, 면서, 며, 위해, 어, 자, 아, 다, 며, 아, 지만, 고...","[다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다]","[., ., ., ., ., ., ., ., ., ., ., ., ., ., ., .]","[사장, 해맞이, 행사, 새해, 각오, 임직원, 선수단, 모두, 이날, 정상, 일출...","[전남, 드래곤즈, 신승재, 구봉산, 구봉산, 전남, 드래곤즈, 신승재, 유상철, ...",...,[],[],4,4,4,4,“새해 정기 받고 올해는 반드시 일내자!”,지역지,medium,425
3,광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,746,"[(광양시, NNP), (는, JX), (오, VV), (는, ETM), (11, ...","[NNP, JX, VV, ETM, SN, NNBC, JX, SN, NNBC, JX,...",325,"[면, 어려워짐, 따라, 고, 위해, 어, 도록, 고, 이뤄, 통해, 어, 고, 맞...","[한다, 다, 다룬다, 진다, 다, 다, 다, 된다]","[., ., ., ., ., ., ., .]","[농업, 인교, 육관, 읍, 회의실, 농업, 대상, 새해, 농업, 실용, 교육, 실...","[광양시, 다래, 이정헌, 전남, 광양시, 농업기술센터]",...,[],[],4,4,5,4,"농업인 역량 강화, 새해 실용교육 실시",지역지,small,290
4,타이완의 크루즈관광객 4000여명이 올해 두 차례에 걸쳐 여수에 입항한다. 전라남도...,978,"[(타이완, NNP), (의, JKG), (크루즈, NNP), (관광객, NNG),...","[NNP, JKG, NNP, NNG, SN, XSN, NNBC, JKS, NNG, ...",402,"[걸쳐, 고, 해, 한다고, 위해, 따라, 해, 해, 고, 위해, 펼쳐왔, 걸쳐, ...","[한다, 다, 다, 다, 한다, 다, 다, 다, 다, 다, 다]","[., ., ., ., ., ., ., ., ., ., .]","[관광객, 올해, 차례, 입항, 올해, 선사, 스타, 급, 관광객, 항, 출발, 여...","[타이완, 크루즈, 여수, 전라남도, 홍콩, 크루즈, 크루즈, 아쿠아, 리우스, 호...",...,[],[],4,4,4,4,타이완 크루즈관광객 4천여명‘전남’온다,지역지,small,370


In [27]:
#- 원문-추출요약문 데이터
ori_ext_df = df[ori_ext]
ori_ext_df.head()

,original_text,ori_char_cnt,ori_pos,only_ori_pos,ori_morph_cnt,ori_EC,ori_EF,ori_SF,ori_NNG,ori_NNP,...,ext_MAG,ext_MAJ,readable,accurate,informative,trustworthy,title,media_sub_type,size,de_ori2ext_morph
0,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,927,"[(전라남도, NNP), (가, JKS), (쌀, NNG), (과잉, NNG), (...","[NNP, JKS, NNG, NNG, NNG, JKO, NNG, XSN, JKB, ...",419,"[위해, 벼, 으면, 해, 나, 해, 해야, 도와, 으나, 다, 며, 도록, 해, ...","[다, 다, 다, 다, 된다, 다, 다, 된다, 한다, 다, 다]","[., ., ., ., ., ., ., ., ., ., .]","[쌀, 과잉, 문제, 근본, 해결, 올해, 시행, 쌀, 생산, 조정, 적극, 추진,...","[전라남도, 전남, 전남, 최향, 전라남도]",...,[],[],4,3,3,3,"논 타작물 재배, 2월 말까지 신청하세요",지역지,small,322
1,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,764,"[(여수시, NNP), (는, JX), (원, NNG), (도심, NNG), (일대...","[NNP, JX, NNG, NNG, NNG, JKB, NNG, XSV+ETM, NN...",339,"[며, 면, 따라, 면, 어, 통해, 고, 해, 고, 돼, 고, 고, 해, 어, 어...","[다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다]","[., ., ., ., ., ., ., ., ., ., .]","[원, 도심, 일대, 추진, 컬러, 빌리지, 사업, 지난해, 말, 마무리, 색채, ...",[여수시],...,"[알록달록, 함께]",[],4,4,5,4,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성",지역지,small,249
2,전남드래곤즈(사장 신승재)는 지난 4일 구봉산 해맞이 행사를 통해 새해 각오를 다졌...,1066,"[(전남, NNP), (드래곤즈, NNP), ((, SSO), (사장, NNG), ...","[NNP, NNP, SSO, NNG, NNP, SSC, JX, VV+ETM, SN,...",454,"[통해, 올라, 며, 면서, 며, 위해, 어, 자, 아, 다, 며, 아, 지만, 고...","[다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다, 다]","[., ., ., ., ., ., ., ., ., ., ., ., ., ., ., .]","[사장, 해맞이, 행사, 새해, 각오, 임직원, 선수단, 모두, 이날, 정상, 일출...","[전남, 드래곤즈, 신승재, 구봉산, 구봉산, 전남, 드래곤즈, 신승재, 유상철, ...",...,"[이내, 다]",[한편],4,4,4,4,“새해 정기 받고 올해는 반드시 일내자!”,지역지,medium,355
3,광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,746,"[(광양시, NNP), (는, JX), (오, VV), (는, ETM), (11, ...","[NNP, JX, VV, ETM, SN, NNBC, JX, SN, NNBC, JX,...",325,"[면, 어려워짐, 따라, 고, 위해, 어, 도록, 고, 이뤄, 통해, 어, 고, 맞...","[한다, 다, 다룬다, 진다, 다, 다, 다, 된다]","[., ., ., ., ., ., ., .]","[농업, 인교, 육관, 읍, 회의실, 농업, 대상, 새해, 농업, 실용, 교육, 실...","[광양시, 다래, 이정헌, 전남, 광양시, 농업기술센터]",...,[],[],4,4,5,4,"농업인 역량 강화, 새해 실용교육 실시",지역지,small,225
4,타이완의 크루즈관광객 4000여명이 올해 두 차례에 걸쳐 여수에 입항한다. 전라남도...,978,"[(타이완, NNP), (의, JKG), (크루즈, NNP), (관광객, NNG),...","[NNP, JKG, NNP, NNG, SN, XSN, NNBC, JKS, NNG, ...",402,"[걸쳐, 고, 해, 한다고, 위해, 따라, 해, 해, 고, 위해, 펼쳐왔, 걸쳐, ...","[한다, 다, 다, 다, 한다, 다, 다, 다, 다, 다, 다]","[., ., ., ., ., ., ., ., ., ., .]","[관광객, 올해, 차례, 입항, 올해, 선사, 스타, 급, 관광객, 항, 출발, 여...","[타이완, 크루즈, 여수, 전라남도, 홍콩, 크루즈, 크루즈, 아쿠아, 리우스, 호...",...,[],[],4,4,4,4,타이완 크루즈관광객 4천여명‘전남’온다,지역지,small,289


In [28]:
#- 추출요약문-생성요약문 데이터
ext_abs_df = df[ext_abs]
ext_abs_df.head()

,converted_ext,ext_char_cnt,ext_pos,only_ext_pos,ext_morph_cnt,ext_EC,ext_EF,ext_SF,ext_NNG,ext_NNP,...,abs_MAG,abs_MAJ,readable,accurate,informative,trustworthy,title,media_sub_type,size,de_ext2abs_morph
0,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,190,"[(전라남도, NNP), (가, JKS), (쌀, NNG), (과잉, NNG), (...","[NNP, JKS, NNG, NNG, NNG, JKO, NNG, XSN, JKB, ...",97,"[위해, 벼, 으면, 해, 해야]","[어요, 예요, 해요]","[., ., .]","[쌀, 과잉, 문제, 근본, 해결, 올해, 시행, 쌀, 생산, 조정, 적극, 추진,...",[전라남도],...,[],[],4,3,3,3,"논 타작물 재배, 2월 말까지 신청하세요",지역지,small,16
1,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,182,"[(여수시, NNP), (는, JX), (원, NNG), (도심, NNG), (일대...","[NNP, JX, NNG, NNG, NNG, JKB, NNG, XSV+ETM, NN...",90,"[며, 따라, 해, 어]","[어요, 어요, 어요]","[., ., .]","[원, 도심, 일대, 추진, 컬러, 빌리지, 사업, 지난해, 말, 마무리, 색채, ...",[여수시],...,[알록달록],[],4,4,5,4,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성",지역지,small,29
2,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,216,"[(임직원, NNG), (과, JC), (선수단, NNG), (모두, NNG), (...","[NNG, JC, NNG, NNG, JX, NNG, NNP, NNG, JKB, VV...",99,"[올라, 며, 어, 자]","[어요, 어요, 어요]","[., ., .]","[임직원, 선수단, 모두, 이날, 정상, 일출, 구단, 목표, 달성, 의지, 사장,...","[구봉산, 신승재, 유상철, 코칭, 스텝, 전남]",...,[],[],4,4,4,4,“새해 정기 받고 올해는 반드시 일내자!”,지역지,medium,70
3,광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,211,"[(광양시, NNP), (는, JX), (오, VV), (는, ETM), (11, ...","[NNP, JX, VV, ETM, SN, NNBC, JX, SN, NNBC, JX,...",100,"[면, 어려워짐, 따라, 고, 위해]","[해요, 어요, 다뤄요]","[., ., .]","[농업, 인교, 육관, 읍, 회의실, 농업, 대상, 새해, 농업, 실용, 교육, 실...","[광양시, 다래]",...,[],[],4,4,5,4,"농업인 역량 강화, 새해 실용교육 실시",지역지,small,65
4,전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급 아쿠아리우스 호가...,257,"[(전라남도, NNP), (는, JX), (올해, NNG), (4, SN), (월,...","[NNP, JX, NNG, SN, NNBC, JC, SN, NNBC, NNP, NN...",113,"[고, 해, 한다고, 위해, 해, 고]","[어요, 어요, 에요]","[., ., .]","[올해, 선사, 스타, 급, 관광객, 항, 출발, 여수, 방문, 신규, 시장, 유치...","[전라남도, 홍콩, 크루즈, 크루즈, 아쿠아, 리우스, 호가, 타이완, 대만, 지룽...",...,[],[],4,4,4,4,타이완 크루즈관광객 4천여명‘전남’온다,지역지,small,81


In [29]:
ori_abs_df.to_csv('aihub_ori_abs.csv',index=False)

In [30]:
ori_ext_df.to_csv('aihub_ori_ext.csv',index=False)

In [31]:
ext_abs_df.to_csv('aihub_ext_abs.csv',index=False)